In [2]:
import paddle
import paddle.nn as nn
import paddle.hub as hub

In [ ]:
paddle.device.set_device('cpu')

# hub model

In [30]:
class ResNet(nn.Layer):
    def __init__(self, name, num_classes=10, pretrained=True):
        super(ResNet, self).__init__()
        
        # hub model from ppclas
        resnet = paddle.hub.load('lyuwenyu/PaddleClas:fix_hub_pretrained_L', name, pretrained=pretrained, )

        # for n, m in resnet.named_children():
        #     print(n, m)
        
        # modified channel 3 -> 1
        resnet.stem[0].conv = nn.Conv2D(1, 64, 7, 2, padding=3,)

        # modified linear 1000 -> 10
        fc = nn.Linear(512, num_classes)
        resnet.fc = fc
        
        # final model
        self.model = resnet
        
    def forward(self, x):
        '''
        '''
        # output = []
        # for m in self.resnet:
        #     x = m(x)
        #     output.append(x)
        # return output

        x = x.reshape([-1, 1, 28, 28])
    
        return self.model(x)

In [31]:
mm = ResNet(name='resnet34')

x = paddle.rand([1, 28 * 28])
output = mm(x)

print(output.shape)
# print([out.shape for out in output])

Using cache found in /root/.cache/paddle/hub/lyuwenyu_PaddleClas_fix_hub_pretrained_L


[2021/08/25 05:51:29] root INFO: unique_endpoints {''}
[2021/08/25 05:51:29] root INFO: Found /root/.paddleclas/weights/ResNet34_pretrained.pdparams
[1, 10]


# train

In [ ]:
import paddle
import paddle.nn as nn
import paddle.vision.transforms as T
from paddle.static import InputSpec

device = paddle.set_device('cpu') # or 'gpu'

net = ResNet(name='resnet34', pretrained=True)

# inputs and labels are not required for dynamic graph.
data = InputSpec([None, 784], 'float32', 'data')
label = InputSpec([None, 1], 'int64', 'label')

model = paddle.Model(net, data, label)
optim = paddle.optimizer.SGD(learning_rate=1e-3, parameters=model.parameters())
model.prepare(optim, nn.CrossEntropyLoss(), paddle.metric.Accuracy())

transform = T.Compose([T.Transpose(), T.Normalize([127.5], [127.5])])

data = paddle.vision.datasets.MNIST(mode='train', transform=transform)
model.fit(data, epochs=2, batch_size=32, verbose=1)


Using cache found in /root/.cache/paddle/hub/lyuwenyu_PaddleClas_fix_hub_pretrained_L


[2021/08/25 05:51:52] root INFO: unique_endpoints {''}
[2021/08/25 05:51:52] root INFO: Found /root/.paddleclas/weights/ResNet34_pretrained.pdparams
item  150/2421 [>.............................] - ETA: 2s - 993us/item

Cache file /root/.cache/paddle/dataset/mnist/train-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-images-idx3-ubyte.gz 
Begin to download


item 8/8 [============================>.] - ETA: 0s - 2ms/item


Download finished
Cache file /root/.cache/paddle/dataset/mnist/train-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-labels-idx1-ubyte.gz 
Begin to download

Download finished


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/2


/paddle/workspace/anaconda3/lib/python3.8/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 1000/1875 [===============>..............] - loss: 0.6011 - acc: 0.7719 - ETA: 12:37 - 866ms/step